In [1]:
from finetune import Workspace as W
import os
import hydra
from omegaconf import OmegaConf
import numpy as np

/home/seolho/anaconda3/envs/urlb/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/seolho/anaconda3/envs/urlb/lib/python3.8/site-packages/torch/utils/tensorboard/__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, "__version__") or LooseVersion(
/home/seolho/anaconda3/envs/urlb/lib/python3.8/site-packages/torch/utils/tensorboard/__init__.py:6: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  ) < LooseVersion("1.15"):


In [2]:
import pathlib
import hydra
hydra.core.global_hydra.GlobalHydra.instance().clear()
config_dir = pathlib.Path('./')
hydra.initialize(config_path=config_dir)

cfg = hydra.compose(config_name='finetune.yaml', overrides=[])


In [3]:
cfg
workspace = W(cfg)

workspace: /home/seolho/workplace/self-supervised-rl/for commit/cic


In [4]:
np.random.uniform(0,1,64).astype(np.float32)

array([4.17021990e-01, 7.20324516e-01, 1.14374816e-04, 3.02332580e-01,
       1.46755889e-01, 9.23385918e-02, 1.86260208e-01, 3.45560730e-01,
       3.96767467e-01, 5.38816750e-01, 4.19194520e-01, 6.85219526e-01,
       2.04452246e-01, 8.78117442e-01, 2.73875929e-02, 6.70467496e-01,
       4.17304814e-01, 5.58689833e-01, 1.40386939e-01, 1.98101491e-01,
       8.00744593e-01, 9.68261600e-01, 3.13424170e-01, 6.92322612e-01,
       8.76389146e-01, 8.94606650e-01, 8.50442126e-02, 3.90547849e-02,
       1.69830427e-01, 8.78142476e-01, 9.83468369e-02, 4.21107620e-01,
       9.57889557e-01, 5.33165276e-01, 6.91877127e-01, 3.15515637e-01,
       6.86500907e-01, 8.34625661e-01, 1.82882771e-02, 7.50144303e-01,
       9.88861084e-01, 7.48165667e-01, 2.80443996e-01, 7.89279342e-01,
       1.03226006e-01, 4.47893530e-01, 9.08595502e-01, 2.93614149e-01,
       2.87775338e-01, 1.30028576e-01, 1.93669572e-02, 6.78835511e-01,
       2.11628109e-01, 2.65546650e-01, 4.91573155e-01, 5.33625446e-02,
      

In [6]:
skill = np.ones(64).astype(np.float32)
workspace.eval(skill)

| eval  | F: 0 | S: 0 | E: 0 | L: 1000 | R: 220.0863 | T: 0:00:00
